<h1>Milestone 1</h1>

<h1>1.⁠ ⁠Understand distribution of contract sizes & lengths </h1>

In [ ]:
txt_folder = "../data/raw/full_contract_txt"

txt_files = os.listdir(txt_folder)
print("Total TXT files:", len(txt_files))

txt_files[:10]

In [ ]:
texts = {}

for file in txt_files:
    path = os.path.join(txt_folder, file)

    try:
        with open(path, "r", encoding="utf-8") as f:
            texts[file] = f.read()
    except:
        with open(path, "r", encoding="latin-1") as f:
            texts[file] = f.read()

print("Loaded", len(texts), "files successfully.")

In [ ]:
stats = []

for filename, text in texts.items():
    words = text.split()
    stats.append({
        "filename": filename,
        "word_count": len(words),
        "char_count": len(text),
        "empty": len(words) == 0
    })

df_stats = pd.DataFrame(stats)
df_stats.head()


In [ ]:
import pandas as pd

# Step 1: Create stats dataframe
stats = []

for filename, text in texts.items():
    words = text.split()
    stats.append({
        "filename": filename,
        "word_count": len(words),
        "char_count": len(text),
        "empty": len(words) == 0
    })

df_stats = pd.DataFrame(stats)
df_stats.head()

# Step 2: Summary statistics
total_contracts = df_stats.shape[0]
avg_words_per_contract = df_stats["word_count"].mean()
median_words_per_contract = df_stats["word_count"].median()
smallest_contract = df_stats["word_count"].min()
largest_contract = df_stats["word_count"].max()

# Step 3: Print results
print(f"Total contracts: {total_contracts}")
print(f"Average words per contract: {avg_words_per_contract:.2f}")
print(f"Median words per contract: {median_words_per_contract}")
print(f"Smallest contract (word count): {smallest_contract}")
print(f"Largest contract (word count): {largest_contract}")

In [ ]:
df_stats[df_stats["empty"] == True]

In [ ]:
# Detect empty or very short contracts
empty_files_count = df_stats[df_stats["empty"] == True].shape[0]
short_files_count = df_stats[df_stats["word_count"] < 100].shape[0]
valid_contracts_count = df_stats[(df_stats["empty"] == False) & (df_stats["word_count"] >= 100)].shape[0]

# Print the results
print(f"Empty files count: {empty_files_count}")
print(f"Short files (< 100 words) count: {short_files_count}")
print(f"Valid contracts eligible for analyzing count: {valid_contracts_count}")

In [ ]:

from collections import Counter
import pandas as pd

# Combine all texts
all_text = " ".join(texts.values())
words = all_text.lower().split()

# Words to remove
remove_list = ["the", "and", "of", "to", "in", "for", "on", "a"]

# Filter words: length > 4 and not in remove_list
legal_words = [w for w in words if len(w) > 4 and w not in remove_list]

# Count frequency
counter = Counter(legal_words)
common_words = counter.most_common(20)

# Convert to dataframe for table display
df_common_words = pd.DataFrame(common_words, columns=["Word", "Frequency"])

# Display header and table
print("Top 20 frequent words")
df_common_words

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from wordcloud import WordCloud
from collections import Counter

# Prepare WordCloud data
all_text = " ".join(texts.values())
words = all_text.lower().split()
remove_list = ["the", "and", "of", "to", "in", "for", "on", "a"]
legal_words = [w for w in words if len(w) > 4 and w not in remove_list]
counter = Counter(legal_words)
common_words = counter.most_common(20)
labels, counts = zip(*common_words)
wc = WordCloud(width=800, height=400, background_color="white").generate(all_text)

# Prepare stats
mean_wc = df_stats["word_count"].mean()
median_wc = df_stats["word_count"].median()

#  subplots
fig, axes = plt.subplots(3, 2, figsize=(18, 18))

# 1. Histogram
axes[0,0].hist(df_stats["word_count"], bins=40, color="#1f77b4", edgecolor="black", alpha=0.7)
axes[0,0].set_title("Histogram – Contract Length (Word Count)", fontsize=14, fontweight="bold")
axes[0,0].set_xlabel("Word Count")
axes[0,0].set_ylabel("Frequency")
axes[0,0].grid(axis="y", linestyle="--", alpha=0.6)
axes[0,0].axvline(mean_wc, color='red', linestyle='dashed', linewidth=1.5, label=f'Mean: {mean_wc:.0f}')
axes[0,0].axvline(median_wc, color='green', linestyle='dashed', linewidth=1.5, label=f'Median: {median_wc:.0f}')
axes[0,0].legend()

# 2. Boxplot
boxprops = dict(facecolor="#1f77b4", color="#1f77b4")
medianprops = dict(color="orange", linewidth=2)
meanprops = dict(marker="o", markerfacecolor="red", markersize=8)
axes[0,1].boxplot(df_stats["word_count"], vert=False, patch_artist=True,
                  boxprops=boxprops, medianprops=medianprops,
                  showmeans=True, meanprops=meanprops)
axes[0,1].set_title("Boxplot - Contract Text Length Distribution", fontsize=14, fontweight="bold")
axes[0,1].set_xlabel("Word Count")
axes[0,1].grid(axis="x", linestyle="--", alpha=0.7)
axes[0,1].text(mean_wc + 5, 1.1, f'Mean: {mean_wc:.0f}', color='red', fontsize=10)
axes[0,1].text(median_wc + 5, 1.1, f'Median: {median_wc:.0f}', color='orange', fontsize=10)

# 3. WordCloud
axes[1,0].imshow(wc, interpolation="bilinear")
axes[1,0].axis("off")
axes[1,0].set_title("WordCloud – Common Clause Keywords", fontsize=14, fontweight="bold")

# 4. Bar chart
axes[1,1].bar(labels, counts, color="#2ca02c")
axes[1,1].set_xticklabels(labels, rotation=75)
axes[1,1].set_title("Top 20 Most Frequent Legal Keywords", fontsize=14, fontweight="bold")

# 5. Scatter plot (spanning bottom row)
fig.delaxes(axes[2,1])  # Remove unused subplot
axes[2,0].scatter(df_stats["char_count"], df_stats["word_count"], color="#ff7f0e", alpha=0.7)
axes[2,0].set_xlabel("Character Count (File Size Approx.)")
axes[2,0].set_ylabel("Word Count")
axes[2,0].set_title("Scatter Plot – File Size vs Word Count", fontsize=14, fontweight="bold")

plt.tight_layout()
plt.show()

<h1> 🔹 TEXT CLEANING  </h1>

In [ ]:
import os
import re


def clean_text(text):
    # Remove page headers/footers
    text = re.sub(r"Page\s*\d+|\f", " ", text)

    # Remove non-ASCII characters
    text = re.sub(r"[^\x00-\x7F]+", " ", text)

    # Normalize whitespace
    text = re.sub(r"\s+", " ", text)

    # Remove repeated line breaks
    text = re.sub(r'\n+', '\n', text)

    # Remove noisy characters (keep letters, numbers, punctuation)
    text = re.sub(r"[^a-zA-Z0-9.,;:()\-\/\n ]", "", text)

    # Fix broken hyphenation (e.g., "exam-\nple" -> "example")
    text = re.sub(r"-\s*\n\s*", "", text)

    # Strip extra spaces
    cleaned = text.strip()

    return cleaned

# -----------------------------
# Clean ALL files in input folder
# -----------------------------
input_folder = "../data/raw/full_contract_txt"
output_folder = "../data/transformed"
os.makedirs(output_folder, exist_ok=True)

for filename in os.listdir(input_folder):
    if filename.endswith(".txt"):
        raw_path = os.path.join(input_folder, filename)

        # Read raw file
        with open(raw_path, "r", encoding="utf-8") as f:
            text = f.read()

        # Clean text using combined function
        cleaned_text = clean_text(text)

        # Save cleaned file
        cleaned_filename = filename.replace(".txt", "_cleaned.txt")
        out_path = os.path.join(output_folder, cleaned_filename)
        with open(out_path, "w", encoding="utf-8") as f:
            f.write(cleaned_text)

print("✔ All cleaned files saved to data/transformed/")


<h1>Sentence Splitting & Chunking with Overlap Chunk Strategy We split into: Batch size 12 gb ram ## FInal (Only chunks NO emneddings )</h1>

In [ ]:
import os
import json
import matplotlib.pyplot as plt
import textwrap
from langchain_text_splitters import RecursiveCharacterTextSplitter


# 1️ PATHS
input_folder = r"K:\Python\ClauseAI\data\transformed"
chunk_folder = r"K:\Python\ClauseAI\dataset\chunks"

os.makedirs(chunk_folder, exist_ok=True)

print("✓ Paths ready")

#
# 2️CHUNKING FUNCTION (SENTENCE-AWARE + OVERLAP)

def chunk_text(text, chunk_size=1000, chunk_overlap=200):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " "]
    )
    return splitter.split_text(text)


# 3.PROCESS ALL CONTRACTS

files = sorted([f for f in os.listdir(input_folder) if f.endswith("_cleaned.txt")])

if not files:
    raise FileNotFoundError("❌ No cleaned files found")

for idx, fname in enumerate(files, start=1):
    path = os.path.join(input_folder, fname)

    with open(path, "r", encoding="utf-8") as f:
        text = f.read()

    # ---- Chunking
    chunks = chunk_text(text)

    # ---- Save chunks
    chunk_out = f"contract_{idx:03d}_chunks.json"
    with open(os.path.join(chunk_folder, chunk_out), "w", encoding="utf-8") as f:
        json.dump(chunks, f, indent=2)

    print(f"✓ Processed → {chunk_out}")


# 4️ PREVIEW ONE CHUNK FILE

sample_file = os.listdir(chunk_folder)[0]
with open(os.path.join(chunk_folder, sample_file), "r", encoding="utf-8") as f:
    chunks = json.load(f)

print("\n📄 Preview")
print("File:", sample_file)
print("Total chunks:", len(chunks))
print(textwrap.shorten(chunks[0], 500))


# 5 VISUALIZATION – CHUNK LENGTH DISTRIBUTION

chunk_lengths = []
for f_name in os.listdir(chunk_folder):
    with open(os.path.join(chunk_folder, f_name), "r", encoding="utf-8") as f:
        chunk_lengths.extend([len(c) for c in json.load(f)])

plt.figure(figsize=(10, 5))
plt.hist(chunk_lengths, bins=40)
plt.title("Chunk Length Distribution")
plt.xlabel("Characters")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()


# 6️ VISUALIZATION – OVERLAP CHECK

overlaps = []
for f_name in os.listdir(chunk_folder):
    with open(os.path.join(chunk_folder, f_name), "r", encoding="utf-8") as f:
        file_chunks = json.load(f)

    for i in range(1, len(file_chunks)):
        prev = file_chunks[i - 1][-200:]
        curr = file_chunks[i][:200]
        overlaps.append(len(set(prev) & set(curr)))

plt.figure(figsize=(10, 5))
plt.plot(overlaps)
plt.title("Overlap Check")
plt.xlabel("Chunk Index")
plt.ylabel("Overlap (approx)")
plt.grid(True)
plt.show()

print("\n✅ Sentence-aware chunking pipeline complete (NO embeddings)")


<h1>Chunk Embeddings & Vector Normalization using #Sentence transformer </h1>

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 1️⃣ IMPORTS & PATHS
chunk_folder = r"K:\Python\ClauseAI\dataset\chunks"
embed_folder = r"K:\Python\ClauseAI\dataset\embeddings"

os.makedirs(embed_folder, exist_ok=True)

print("✓ Paths ready")

# 2️⃣ LOAD SENTENCE TRANSFORMER MODEL
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)

EMBED_DIM = model.get_sentence_embedding_dimension()
print(f"✓ Model loaded: {MODEL_NAME} ({EMBED_DIM} dims)")

# 3️⃣ EMBEDDING FUNCTION
def generate_embeddings(chunks, batch_size=64):
    return model.encode(
        chunks,
        batch_size=batch_size,
        show_progress_bar=False,
        normalize_embeddings=False
    )

# 4️⃣ PROCESS ALL CHUNK FILES (FIRST 20 FOR DEMO)
chunk_files = sorted(
    [f for f in os.listdir(chunk_folder) if f.endswith("_chunks.json")]
)[:20]

if not chunk_files:
    raise FileNotFoundError("❌ No chunk files found")

all_norms = []

for fname in chunk_files:
    with open(os.path.join(chunk_folder, fname), "r", encoding="utf-8") as f:
        chunks = json.load(f)

    embeddings = generate_embeddings(chunks)

    records = []
    for text, emb in zip(chunks, embeddings):
        norm = float(np.linalg.norm(emb))
        all_norms.append(norm)

        records.append({
            "text": text,
            "embedding": emb.tolist(),
            "norm": norm
        })

    out_name = fname.replace("_chunks.json", "_embeddings.json")
    with open(os.path.join(embed_folder, out_name), "w", encoding="utf-8") as f:
        json.dump(records, f, indent=2)

    print(f"✓ Embedded → {out_name}")

# 5️⃣ PREVIEW ONE EMBEDDING FILE
sample_file = sorted(os.listdir(embed_folder))[0]
with open(os.path.join(embed_folder, sample_file), "r", encoding="utf-8") as f:
    sample_data = json.load(f)

print("\n📄 Preview Embedding File")
print("File:", sample_file)
print("Total vectors:", len(sample_data))
print("Embedding length:", len(sample_data[0]["embedding"]))

# 6️⃣ VECTOR LENGTH CHECK
lengths = {len(item["embedding"]) for item in sample_data}
print("\n📐 Vector Length Check")
print("Unique vector lengths:", lengths)

# 7️⃣ VISUALIZATION – EMBEDDING NORM DISTRIBUTION
plt.figure(figsize=(10, 5))
plt.hist(all_norms, bins=10)   # FIXED: safe bin count
plt.title("Embedding Norm Distribution")
plt.xlabel("Vector Norm")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

# 8️⃣ SANITY CHECK – SIMILARITY (COSINE & DOT PRODUCT)
v1 = np.array(sample_data[0]["embedding"])
v2 = np.array(sample_data[1]["embedding"])

cos_sim = cosine_similarity([v1], [v2])[0][0]
dot_sim = np.dot(v1, v2)

print("\n🧪 Similarity Sanity Check")
print(f"Cosine Similarity : {cos_sim:.4f}")
print(f"Dot Product      : {dot_sim:.4f}")

print("\n✅ Embedding pipeline completed successfully")


<h1 > Chunk Embeddings & Vector Normalization </h1>

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 1️⃣ IMPORTS & PATHS
chunk_folder = r"K:\Python\ClauseAI\dataset\chunks"
embed_folder = r"K:\Python\ClauseAI\dataset\embeddings"

os.makedirs(embed_folder, exist_ok=True)

print("✓ Paths ready")

# 2️⃣ LOAD SENTENCE TRANSFORMER MODEL
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)

EMBED_DIM = model.get_sentence_embedding_dimension()
print(f"✓ Model loaded: {MODEL_NAME} ({EMBED_DIM} dims)")

# 3️⃣ EMBEDDING FUNCTION
def generate_embeddings(chunks, batch_size=64):
    return model.encode(
        chunks,
        batch_size=batch_size,
        show_progress_bar=True,
        normalize_embeddings=False
    )

# 4️⃣ PROCESS ALL CHUNK FILES
chunk_files = sorted(
    f for f in os.listdir(chunk_folder) if f.endswith("_chunks.json")
)

if not chunk_files:
    raise FileNotFoundError("❌ No chunk files found")

all_norms = []
total_chunks = 0

for fname in chunk_files:
    with open(os.path.join(chunk_folder, fname), "r", encoding="utf-8") as f:
        chunks = json.load(f)

    total_chunks += len(chunks)

    embeddings = generate_embeddings(chunks)

    records = []
    for text, emb in zip(chunks, embeddings):
        norm = float(np.linalg.norm(emb))
        all_norms.append(norm)

        records.append({
            "text": text,
            "embedding": emb.tolist(),
            "norm": norm
        })

    out_name = fname.replace("_chunks.json", "_embeddings.json")
    with open(os.path.join(embed_folder, out_name), "w", encoding="utf-8") as f:
        json.dump(records, f, indent=2)

    print(f"✓ Embedded → {out_name} ({len(chunks)} chunks)")

print(f"\n🔢 TOTAL CHUNKS EMBEDDED: {total_chunks}")

# 5️⃣ PREVIEW ONE EMBEDDING FILE
sample_file = sorted(os.listdir(embed_folder))[0]
with open(os.path.join(embed_folder, sample_file), "r", encoding="utf-8") as f:
    sample_data = json.load(f)

print("\n📄 Preview Embedding File")
print("File:", sample_file)
print("Total vectors:", len(sample_data))
print("Embedding length:", len(sample_data[0]["embedding"]))

# 6️⃣ VECTOR LENGTH CHECK
lengths = {len(item["embedding"]) for item in sample_data}
print("\n📐 Vector Length Check")
print("Unique vector lengths:", lengths)

# 7️⃣ VISUALIZATION – EMBEDDING NORM DISTRIBUTION
plt.figure(figsize=(10, 5))
plt.hist(all_norms, bins=10)
plt.title("Embedding Norm Distribution")
plt.xlabel("Vector Norm")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()

# 8️⃣ SANITY CHECK – SIMILARITY (COSINE & DOT PRODUCT)
v1 = np.array(sample_data[0]["embedding"])
v2 = np.array(sample_data[1]["embedding"])

cos_sim = cosine_similarity([v1], [v2])[0][0]
dot_sim = np.dot(v1, v2)

print("\n🧪 Similarity Sanity Check")
print(f"Cosine Similarity : {cos_sim:.4f}")
print(f"Dot Product      : {dot_sim:.4f}")

print("\n✅ All contract chunks embedded successfully")


#Next part - visualization

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity


# 1️ PATHS

embed_folder = r"K:\Python\ClauseAI\dataset\embeddings"

print("✓ Using embeddings from:", embed_folder)

# 2️ LOAD ONE EMBEDDING FILE (PREVIEW)
embedding_files = sorted(
    f for f in os.listdir(embed_folder) if f.endswith("_embeddings.json")
)

if not embedding_files:
    raise FileNotFoundError("❌ No embedding files found")

sample_file = embedding_files[0]

with open(os.path.join(embed_folder, sample_file), "r", encoding="utf-8") as f:
    sample_data = json.load(f)

print("\n📄 Preview One Embedding File")
print("File:", sample_file)
print("Total vectors:", len(sample_data))
print("Embedding dimension:", len(sample_data[0]["embedding"]))
print("Sample text snippet:")
print(sample_data[0]["text"][:300], "...")


# 3️ VECTOR LENGTH CHECK

vector_lengths = set()

for fname in embedding_files:
    with open(os.path.join(embed_folder, fname), "r", encoding="utf-8") as f:
        data = json.load(f)
        for item in data:
            vector_lengths.add(len(item["embedding"]))

print("\n📐 Vector Length Check")
print("Unique vector lengths across dataset:", vector_lengths)


# 4️ COLLECT ALL EMBEDDING NORMS

all_norms = []

for fname in embedding_files:
    with open(os.path.join(embed_folder, fname), "r", encoding="utf-8") as f:
        data = json.load(f)
        for item in data:
            all_norms.append(item["norm"])


# 5️VISUALIZATION – EMBEDDING NORM DISTRIBUTION

plt.figure(figsize=(10, 5))
plt.hist(all_norms, bins=10)
plt.title("Embedding Norm Distribution")
plt.xlabel("Vector Norm")
plt.ylabel("Frequency")
plt.grid(True)
plt.show()


# 6️ SANITY CHECK – SIMILARITY (COSINE & DOT PRODUCT)

vec1 = np.array(sample_data[0]["embedding"])
vec2 = np.array(sample_data[1]["embedding"])

cos_sim = cosine_similarity([vec1], [vec2])[0][0]
dot_sim = np.dot(vec1, vec2)

print("\n🧪 Similarity Sanity Check")
print(f"Cosine Similarity : {cos_sim:.4f}")
print(f"Dot Product      : {dot_sim:.4f}")

print("\n✅ Embedding inspection & sanity checks complete")


In [ ]:
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
!pip install -U pinecone-client sentence-transformers matplotlib


In [ ]:
!pip uninstall -y pinecone-client


In [ ]:
!pip install -U pinecone sentence-transformers matplotlib


In [ ]:
# =============================================================
# PINECONE: CONNECT TO INDEX, UPSERT VECTORS, AND QUERY
# =============================================================

import os
import json
import uuid
import matplotlib.pyplot as plt
from sentence_transformers import SentenceTransformer
from pinecone import Pinecone, ServerlessSpec

# PINECONE INITIALIZATION
# -------------------------------------------------------------
PINECONE_API_KEY = "pcsk_6jbLBU_DxNgioCN5BHBNM6x3S2Gd9WMY3DDVnruCFBSsEa7efABnmRWydJhEn4itJDVfG"
PINECONE_ENV = "us-east-1"
INDEX_NAME = "pinevs"

# Initialize client
pc = Pinecone(api_key=PINECONE_API_KEY)
print("✓ Pinecone client initialized")

# CONNECT TO INDEX
EMBED_DIM = 384  # MiniLM dimension

if INDEX_NAME not in pc.list_indexes().names():
    print(" Creating Pinecone index...")
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBED_DIM,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region=PINECONE_ENV)
    )
else:
    print("✓ Index already exists")

index = pc.Index(INDEX_NAME)
print(f"✓ Connected to index: {INDEX_NAME}")

# 3️.LOAD EMBEDDINGS
embed_folder = "/content/drive/MyDrive/ClauseAI/dataset/embeddings"
embedding_files = sorted(f for f in os.listdir(embed_folder) if f.endswith("_embeddings.json"))

if not embedding_files:
    raise FileNotFoundError("❌ No embedding files found")
print(f"✓ Found {len(embedding_files)} embedding files")

# 4️. BUILD VECTORS FOR UPSERT
vectors = []
MAX_CONTRACTS = 20

for fname in embedding_files:
    with open(os.path.join(embed_folder, fname), "r", encoding="utf-8") as f:
        records = json.load(f)

    for i, rec in enumerate(records, start=1):
        vector_id = f"demo-visible-vector-{i}"  # readable in web UI
        vectors.append({
            "id": vector_id,
            "values": rec["embedding"],  # must be 384-dim
            "metadata": {"text": rec["text"], "source_file": fname}
        })

        if len(vectors) >= MAX_CONTRACTS:
            break
    if len(vectors) >= MAX_CONTRACTS:
        break

print(f"✓ Prepared {len(vectors)} vectors for upsert")

# 5️⃣ UPSERT VECTORS
index.upsert(vectors=vectors, namespace="__default__")
print("✅ Vector upserted successfully (check web UI under __default__)")

# 6️⃣ TEST QUERY (SEMANTIC SEARCH)
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
model = SentenceTransformer(MODEL_NAME)

query_text = "termination clause and contract cancellation"
query_embedding = model.encode(query_text).tolist()

results = index.query(
    vector=query_embedding,
    top_k=10,
    include_metadata=True,
    namespace="__default__"
)

# 7️⃣ PRINT TOP-10 RESULTS
print("\n🔍 QUERY:", query_text)
print("\n🏆 TOP 10 MATCHES:\n")
scores = []

for i, match in enumerate(results["matches"], start=1):
    scores.append(match["score"])
    print(f"{i}. ID: {match['id']}, Score: {match['score']:.4f}")
    print(f"   Text: {match['metadata']['text'][:300]}...")
    print("-" * 80)

# 8️⃣ VISUALIZATION
plt.figure(figsize=(8, 4))
plt.bar(range(1, len(scores) + 1), scores)
plt.xlabel("Rank")
plt.ylabel("Similarity Score")
plt.title("Top-K Pinecone Similarity Scores")
plt.grid(True)
plt.show()

print("\n✅ Pipeline completed. Visit Pinecone web UI to see vectors!")

In [ ]:
# =============================================================
# RAG SEARCH WRAPPER – CLAUSEAI (GOOGLE COLAB VERSION)
# =============================================================

# 0️⃣ MOUNT GOOGLE DRIVE
from google.colab import drive
drive.mount("/content/drive")

# -------------------------------------------------------------
# 1️⃣ IMPORTS
# -------------------------------------------------------------
import os
import json
import re
import numpy as np
import matplotlib.pyplot as plt

from sentence_transformers import SentenceTransformer
from pinecone import Pinecone

# -------------------------------------------------------------
# 2️⃣ CONFIG
# -------------------------------------------------------------
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
INDEX_NAME = "pinevs"

PINECONE_API_KEY = "pcsk_6jbLBU_DxNgioCN5BHBNM6x3S2Gd9WMY3DDVnruCFBSsEa7efABnmRWydJhEn4itJDVfG"

RAG_JSON_FILE = "/content/drive/MyDrive/ClauseAI/data/RAG_result/"
os.makedirs(RAG_JSON_FILE, exist_ok=True)

TOP_K = 5

# -------------------------------------------------------------
# 3️⃣ INIT MODEL & PINECONE
# -------------------------------------------------------------
model = SentenceTransformer(MODEL_NAME)

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index(INDEX_NAME)

print("✓ Model & Pinecone index ready")

# -------------------------------------------------------------
# 4️⃣ HELPER: EMBED QUERY
# -------------------------------------------------------------
def embed_query(query: str):
    """
    Convert query text to embedding
    """
    emb = model.encode(query)
    return emb.tolist()

# -------------------------------------------------------------
# 5️⃣ RAG SEARCH FUNCTION
# -------------------------------------------------------------
def rag_search(query, top_k=TOP_K):
    query_embedding = embed_query(query)

    results = index.query(
        vector=query_embedding,
        top_k=top_k,
        include_metadata=True
    )

    return results["matches"]

# -------------------------------------------------------------
# 6️⃣ KEYWORD HIGHLIGHTING
# -------------------------------------------------------------
def highlight_text(text, keywords):
    for kw in keywords:
        pattern = re.compile(rf"\b({re.escape(kw)})\b", re.IGNORECASE)
        text = pattern.sub(r"**\1**", text)
    return text

# -------------------------------------------------------------
# 7️⃣ PRETTY PRINT RESULTS
# -------------------------------------------------------------
def pretty_print(matches, query):
    print("\n🔍 QUERY:")
    print(query)
    print("\n📄 TOP MATCHES:\n")

    scores = []
    keywords = query.lower().split()

    for i, match in enumerate(matches, start=1):
        score = match["score"]
        text = match["metadata"]["text"]

        scores.append(score)
        highlighted = highlight_text(text, keywords)

        print(f"{i}. Similarity Score: {score:.4f}")
        print(highlighted[:500] + "...")
        print("-" * 80)

    return scores

# -------------------------------------------------------------
# 8️⃣ VISUALIZE SIMILARITY SCORES
# -------------------------------------------------------------
def visualize_scores(scores):
    plt.figure(figsize=(8, 4))
    plt.bar(range(1, len(scores) + 1), scores)
    plt.xlabel("Rank")
    plt.ylabel("Similarity Score")
    plt.title("RAG Top-K Similarity Scores")
    plt.grid(True)
    plt.show()

# -------------------------------------------------------------
# 9️⃣ SAVE RESULTS TO GOOGLE DRIVE (JSON)
# -------------------------------------------------------------
def save_results(query, matches):
    output = {
        "query": query,
        "results": [
            {
                "rank": i + 1,
                "score": m["score"],
                "text": m["metadata"]["text"],
                "source": m["metadata"].get("source_file", "unknown")
            }
            for i, m in enumerate(matches)
        ]
    }

    file_path = os.path.join(
        RAG_JSON_FILE,
        f"rag_result_{abs(hash(query))}.json"
    )

    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(output, f, indent=2)

    print(f"\n💾 Results saved to Google Drive:\n{file_path}")

# -------------------------------------------------------------
# 🔟 TEST WITH A REAL LEGAL QUERY
# -------------------------------------------------------------
if __name__ == "__main__":
    test_query = "termination clause and contract cancellation rights"

    matches = rag_search(test_query)
    scores = pretty_print(matches, test_query)
    visualize_scores(scores)
    save_results(test_query, matches)

    print("\n✅ RAG retrieval completed successfully")


Agent Framework Setup + Standard Output Schema


In [ ]:
import json
import re
from sentence_transformers import SentenceTransformer

In [ ]:
MODEL_NAME = "sentence-transformers/all-MiniLM-L6-v2"
embedding_model = SentenceTransformer(MODEL_NAME)

In [ ]:
AGENT_OUTPUT_SCHEMA = {
    "clause_type": "",
    "extracted_clauses": [],
    "risk_level": "unknown",
    "confidence": 0.0,
    "evidence": []
}

3Base Agent Class (Embedding + Rule-Based)


In [ ]:
class BaseAgent:
    def __init__(self, agent_name: str, keywords: list):
        self.agent_name = agent_name
        self.keywords = keywords
        self.keyword_embeddings = embedding_model.encode(keywords)

    def run(self, context_text: str) -> dict:
        sentences = re.split(r'(?<=[.!?])\s+', context_text.strip())
        extracted = []
        evidence = []

        for sent in sentences:
            sent_emb = embedding_model.encode(sent)
            similarities = [
                self._cosine_similarity(sent_emb, kw_emb)
                for kw_emb in self.keyword_embeddings
            ]

            if max(similarities) > 0.6:  # similarity threshold
                extracted.append(sent)
                evidence.append(sent)

        risk = self._assess_risk(len(extracted))
        confidence = round(min(1.0, len(extracted) / 5), 2)

        return {
            "extracted_clauses": extracted,
            "risk_level": risk,
            "confidence": confidence,
            "evidence": evidence
        }

    @staticmethod
    def _cosine_similarity(a, b):
        return (a @ b) / ((a @ a) ** 0.5 * (b @ b) ** 0.5)

    @staticmethod
    def _assess_risk(count):
        if count == 0:
            return "low"
        elif count <= 2:
            return "medium"
        else:
            return "high"


JSON Validation Helper


In [ ]:
def validate_agent_output(raw_output: dict, clause_type: str) -> dict:
    validated = AGENT_OUTPUT_SCHEMA.copy()
    validated.update(raw_output)
    validated["clause_type"] = clause_type

    validated.setdefault("extracted_clauses", [])
    validated.setdefault("risk_level", "unknown")
    validated.setdefault("confidence", 0.0)
    validated.setdefault("evidence", [])

    return validated

Example Dummy Agent (LEGAL – Termination)


In [ ]:
LEGAL_TERMINATION_KEYWORDS = [
    "termination",
    "terminate",
    "cancellation",
    "breach",
    "notice period"
]

legal_agent = BaseAgent(
    agent_name="Legal Agent",
    keywords=LEGAL_TERMINATION_KEYWORDS
)

In [ ]:
dummy_contract = """
This agreement may be terminated by either party with 30 days written notice.
Payment obligations shall survive termination.
In case of breach, the contract may be cancelled immediately.
"""

raw_output = legal_agent.run(dummy_contract)

validated_output = validate_agent_output(
    raw_output,
    clause_type="Termination"
)

print(json.dumps(validated_output, indent=2))

Legal agent

In [ ]:
# =============================================================
# LEGAL AGENT — MISTRAL-7B-INSTRUCT v0.3
# =============================================================

# -----------------------------
# 0️⃣ INSTALL DEPENDENCIES
# -----------------------------
!pip install -q transformers accelerate sentencepiece

# -----------------------------
# 1️⃣ IMPORTS
# -----------------------------
import os
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import drive

# -----------------------------
# 2️⃣ MOUNT GOOGLE DRIVE
# -----------------------------
drive.mount('/content/drive')

# -----------------------------
# 3️⃣ LOAD RAG CONTEXT
# -----------------------------
RAG_JSON_FILE = "/content/drive/MyDrive/ClauseAI/data/RAG_result/rag_result_6520704210135594280.json"

with open(RAG_JSON_FILE, "r", encoding="utf-8") as f:
    legal_context = json.load(f)

print(f"✅ Loaded {len(legal_context)} RAG chunks")

combined_text = "\n\n".join(
    c["text"] for c in legal_context if "text" in c
)

print("🔹 Combined text preview:\n", combined_text[:300])

# -----------------------------
# 4️⃣ LEGAL AGENT PROMPT
# -----------------------------
LEGAL_AGENT_PROMPT = """
You are a Legal Contract Analysis Agent.

Tasks:
1. Identify legal clauses (Termination, Governing Law, Jurisdiction).
2. Extract exact clause text.
3. Assess legal risk (low/medium/high).
4. Provide confidence score (0.0 - 1.0).

Return ONLY valid JSON:
{
  "clause_type": "Legal Analysis",
  "extracted_clauses": [],
  "risk_level": "",
  "confidence": 0.0,
  "evidence": []
}
"""

# -----------------------------
# 5️⃣ LOAD MISTRAL (NO 4-BIT)
# -----------------------------
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    torch_dtype=torch.float16,
    device_map="auto"
)

model.eval()
print("✅ Mistral model loaded successfully")

# -----------------------------
# 6️⃣ BASE AGENT
# -----------------------------
class BaseAgent:
    def __init__(self, model, tokenizer, system_prompt):
        self.model = model
        self.tokenizer = tokenizer
        self.system_prompt = system_prompt

    def run(self, input_text):
        prompt = f"""
<s>[INST]
{self.system_prompt}

CONTRACT TEXT:
{input_text}
[/INST]
"""
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=4096
        ).to(self.model.device)

        with torch.no_grad():
            output = self.model.generate(
                **inputs,
                max_new_tokens=1200,
                do_sample=False
            )

        return self.tokenizer.decode(
            output[0],
            skip_special_tokens=True
        ).strip()

# -----------------------------
# 7️⃣ RUN LEGAL AGENT
# -----------------------------
legal_agent = BaseAgent(
    model=model,
    tokenizer=tokenizer,
    system_prompt=LEGAL_AGENT_PROMPT
)

raw_output = legal_agent.run(combined_text)
print("\n🔹 RAW MODEL OUTPUT (TRUNCATED):\n", raw_output[:800])

# -----------------------------
# 8️⃣ SAFE JSON EXTRACTION
# -----------------------------
def extract_json(text: str) -> dict:
    text = re.sub(r"```json|```", "", text)

    stack = []
    start = None
    for i, ch in enumerate(text):
        if ch == "{":
            if not stack:
                start = i
            stack.append("{")
        elif ch == "}":
            if stack:
                stack.pop()
                if not stack:
                    json_str = text[start:i+1]
                    break
    else:
        raise ValueError("❌ No JSON found")

    data = json.loads(json_str)

    data.setdefault("clause_type", "Legal Analysis")
    data.setdefault("extracted_clauses", [])
    data.setdefault("risk_level", "unknown")
    data.setdefault("confidence", 0.0)
    data.setdefault("evidence", [])

    return data

validated_output = extract_json(raw_output)
print("✅ Legal JSON extracted")

# -----------------------------
# 9️⃣ SAVE OUTPUT
# -----------------------------
OUTPUT_FILE = "/content/drive/MyDrive/ClauseAI/data/Agent/legal_agent_output.json"
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(validated_output, f, indent=2, ensure_ascii=False)

print(f"\n✅ Output saved at:\n{OUTPUT_FILE}")

</h1> COMPLIANCE AGENT <h1>

In [ ]:
# =============================================================
# COMPLIANCE AGENT — MISTRAL-7B-INSTRUCT v0.3 — FULL WORKING CODE
# =============================================================

# -----------------------------
# 0️⃣ INSTALL DEPENDENCIES
# -----------------------------
!pip install -q transformers accelerate sentencepiece bitsandbytes

# -----------------------------
# 1️⃣ IMPORTS
# -----------------------------
import os
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import drive

# -----------------------------
# 2️⃣ MOUNT GOOGLE DRIVE
# -----------------------------
drive.mount('/content/drive')

# -----------------------------
# 3️⃣ LOAD COMPLIANCE-RELATED CONTEXT (RAG)
# -----------------------------
RAG_JSON_FILE = "/content/drive/MyDrive/ClauseAI/data/RAG_result/rag_result_6520704210135594280.json"

with open(RAG_JSON_FILE, "r", encoding="utf-8") as f:
    compliance_context = json.load(f)

print(f"✅ Loaded {len(compliance_context)} RAG chunks")

# -----------------------------
# 4️⃣ COMBINE CONTEXT
# -----------------------------
combined_text = "\n\n".join(c["text"] for c in compliance_context if "text" in c)
print("🔹 Combined compliance text preview:\n", combined_text[:300])

# -----------------------------
# 5️⃣ COMPLIANCE AGENT PROMPT
# -----------------------------
COMPLIANCE_AGENT_PROMPT = """
You are a Compliance Risk Analysis Agent.

Your task:
1. Identify compliance-related clauses:
- Data protection
- Regulatory requirements
- Audits & reporting
- GDPR
- SOC2
- ISO
- HIPAA

2. Extract exact compliance obligations
3. Assess compliance risk (low/medium/high)
4. Provide confidence score

Return ONLY valid JSON in this format:
{
  "clause_type": "Compliance Analysis",
  "extracted_clauses": [],
  "risk_level": "",
  "confidence": 0.0,
  "evidence": []
}
"""

# -----------------------------
# 6️⃣ LOAD MISTRAL-7B-INSTRUCT SAFELY
# -----------------------------
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto"
)

model.eval()
print("✅ Mistral model loaded safely with 4-bit quantization")

# -----------------------------
# 7️⃣ BASE AGENT CLASS
# -----------------------------
class BaseAgent:
    def __init__(self, model, tokenizer, system_prompt):
        self.model = model
        self.tokenizer = tokenizer
        self.system_prompt = system_prompt

    def run(self, input_text):
        prompt = f"""
<s>[INST]
{self.system_prompt}

CONTRACT TEXT:
{input_text}
[/INST]
"""
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=4096
        ).to(self.model.device)

        with torch.no_grad():
            output = self.model.generate(
                **inputs,
                max_new_tokens=1200,
                do_sample=False
            )

        decoded = self.tokenizer.decode(output[0], skip_special_tokens=True)
        return decoded.strip()

# -----------------------------
# 8️⃣ RUN COMPLIANCE AGENT
# -----------------------------
compliance_agent = BaseAgent(
    model=model,
    tokenizer=tokenizer,
    system_prompt=COMPLIANCE_AGENT_PROMPT
)

raw_output = compliance_agent.run(combined_text)
print("\n🔹 RAW MODEL OUTPUT (TRUNCATED):\n", raw_output[:1000])

# -----------------------------
# 9️⃣ SAFE JSON EXTRACTION WITH DEFAULTS
# -----------------------------
def extract_json_with_defaults(text):
    """
    Extract first JSON object from text and fill default fields
    """
    # Remove markdown fences
    text = re.sub(r"```json|```", "", text)

    # Match first JSON object using braces counting
    stack = []
    start_idx = None
    for i, char in enumerate(text):
        if char == '{':
            if not stack:
                start_idx = i
            stack.append('{')
        elif char == '}':
            if stack:
                stack.pop()
                if not stack:
                    json_str = text[start_idx:i+1]
                    break
    else:
        raise ValueError("❌ No JSON object found in text")

    # Load JSON
    data = json.loads(json_str)

    # Set defaults
    data.setdefault("clause_type", "Compliance Analysis")
    data.setdefault("extracted_clauses", [])
    data.setdefault("risk_level", "medium")
    data.setdefault("confidence", 0.8)
    data.setdefault("evidence", [])

    return data

validated_output = extract_json_with_defaults(raw_output)
print("✅ Compliance JSON extracted and validated")

# -----------------------------
# 🔟 SAVE OUTPUT TO DRIVE
# -----------------------------
OUTPUT_FILE = "/content/drive/MyDrive/ClauseAI/data/Agent/compliance_agent_output.json"
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(validated_output, f, indent=2, ensure_ascii=False)

print(f"\n✅ Compliance Agent output saved at:\n{OUTPUT_FILE}")

FINANCE AGENT


In [ ]:
# =============================================================
# FINANCE AGENT — MISTRAL-7B-INSTRUCT v0.3 — FULL WORKING CODE
# =============================================================

# -----------------------------
# 0️⃣ INSTALL DEPENDENCIES
# -----------------------------
!pip install -q transformers accelerate sentencepiece bitsandbytes

# -----------------------------
# 1️⃣ IMPORTS
# -----------------------------
import os
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from google.colab import drive

# -----------------------------
# 2️⃣ MOUNT GOOGLE DRIVE
# -----------------------------
drive.mount('/content/drive')

# -----------------------------
# 3️⃣ LOAD FINANCE-RELATED CONTEXT (RAG)
# -----------------------------
RAG_JSON_FILE = "/content/drive/MyDrive/ClauseAI/data/RAG_result/rag_result_6520704210135594280.json"

with open(RAG_JSON_FILE, "r", encoding="utf-8") as f:
    finance_context = json.load(f)

print(f"✅ Loaded {len(finance_context)} RAG chunks")

# -----------------------------
# 4️⃣ COMBINE CONTEXT
# -----------------------------
combined_text = "\n\n".join(c["text"] for c in finance_context if "text" in c)
print("🔹 Combined finance text preview:\n", combined_text[:300])

# -----------------------------
# 5️⃣ FINANCE AGENT PROMPT
# -----------------------------
FINANCE_AGENT_PROMPT = """
You are a Finance Risk Analysis Agent.

Your task:
1. Identify finance-related clauses:
- Payment terms
- Fees and invoices
- Late fees
- Penalties
- Financial liability

2. Extract exact financial obligations
3. Assess financial risk (low/medium/high)
4. Provide confidence score

Return ONLY valid JSON:
{
  "extracted_clauses": [],
  "risk_level": "",
  "confidence": 0.0,
  "evidence": []
}
"""

# -----------------------------
# 6️⃣ LOAD MISTRAL-7B-INSTRUCT SAFELY
# -----------------------------
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto"
)

model.eval()
print("✅ Mistral model loaded with 4-bit quantization")

# -----------------------------
# 7️⃣ BASE AGENT CLASS
# -----------------------------
class BaseAgent:
    def __init__(self, model, tokenizer, system_prompt):
        self.model = model
        self.tokenizer = tokenizer
        self.system_prompt = system_prompt

    def run(self, input_text):
        prompt = f"""
<s>[INST]
{self.system_prompt}

CONTRACT TEXT:
{input_text}
[/INST]
"""
        inputs = self.tokenizer(
            prompt,
            return_tensors="pt",
            truncation=True,
            max_length=4096
        ).to(self.model.device)

        with torch.no_grad():
            output = self.model.generate(
                **inputs,
                max_new_tokens=1200,
                do_sample=False
            )

        decoded = self.tokenizer.decode(output[0], skip_special_tokens=True)
        return decoded.strip()

# -----------------------------
# 8️⃣ RUN FINANCE AGENT
# -----------------------------
finance_agent = BaseAgent(
    model=model,
    tokenizer=tokenizer,
    system_prompt=FINANCE_AGENT_PROMPT
)

raw_output = finance_agent.run(combined_text)
print("\n🔹 RAW MODEL OUTPUT (TRUNCATED):\n", raw_output[:1000])

# -----------------------------
# 9️⃣ SAFE JSON EXTRACTION WITH DEFAULTS
# -----------------------------
def extract_json_with_defaults(text):
    """
    Extract first JSON object from text and fill default fields
    """
    text = re.sub(r"```json|```", "", text)

    stack = []
    start_idx = None
    for i, char in enumerate(text):
        if char == '{':
            if not stack:
                start_idx = i
            stack.append('{')
        elif char == '}':
            if stack:
                stack.pop()
                if not stack:
                    json_str = text[start_idx:i+1]
                    break
    else:
        raise ValueError("❌ No JSON object found")

    data = json.loads(json_str)

    data.setdefault("extracted_clauses", [])
    data.setdefault("risk_level", "medium")
    data.setdefault("confidence", 0.8)
    data.setdefault("evidence", [])

    return data

validated_output = extract_json_with_defaults(raw_output)
print("✅ Finance JSON extracted and validated")

# -----------------------------
# 🔟 SAVE OUTPUT TO DRIVE
# -----------------------------
OUTPUT_FILE = "/content/drive/MyDrive/ClauseAI/data/Agent/finance_agent_output.json"
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(validated_output, f, indent=2, ensure_ascii=False)

print(f"\n✅ Finance Agent output saved at:\n{OUTPUT_FILE}")

OPERATIONN AGENT


In [ ]:
# =============================================================
# OPERATIONS AGENT — OBLIGATIONS, DELIVERABLES & EXECUTION RISK
# =============================================================

# -----------------------------
# 0️⃣ INSTALL DEPENDENCIES
# -----------------------------
!pip install -q transformers accelerate sentencepiece

# -----------------------------
# 1️⃣ IMPORTS
# -----------------------------
import os
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from google.colab import drive

# -----------------------------
# 2️⃣ MOUNT GOOGLE DRIVE
# -----------------------------
drive.mount("/content/drive")

# -----------------------------
# 3️⃣ LOAD OPERATIONAL CONTEXT
# -----------------------------
RAG_JSON_FILE = "/content/drive/MyDrive/ClauseAI/data/RAG_result/rag_result_6520704210135594280.json"

with open(RAG_JSON_FILE, "r", encoding="utf-8") as f:
    operational_context = json.load(f)

print(f"✅ Loaded {len(operational_context)} RAG chunks")

combined_text = "\n\n".join(
    c["text"] for c in operational_context if "text" in c
)

print("🔹 Combined operations text preview:\n", combined_text[:300])

# -----------------------------
# 4️⃣ OPERATIONS AGENT PROMPT
# -----------------------------
OPERATIONS_AGENT_PROMPT = """
You are an Operations Risk Analysis Agent.

Your task:
1. Identify operational clauses:
- Deliverables
- Timelines
- Milestones
- Service obligations
- Performance standards / SLAs

2. Extract exact obligation text from the contract
3. Assess execution risk (low/medium/high)

Rules:
- Copy text verbatim
- Do NOT infer or summarize
- If no clause exists, return empty array

Return ONLY valid JSON:
{
  "extracted_clauses": [],
  "risk_level": "",
  "confidence": 0.0,
  "evidence": []
}
"""

# -----------------------------
# 5️⃣ LOAD MODEL (STABLE MODE)
# -----------------------------
MODEL_ID = "mistralai/Mistral-7B-Instruct-v0.3"

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",
    torch_dtype=torch.float16
)

model.eval()
print("✅ Model loaded")

# -----------------------------
# 6️⃣ INITIALIZE OPERATIONS AGENT
# -----------------------------
class OperationsAgent:
    def __init__(self, model, tokenizer, prompt):
        self.model = model
        self.tokenizer = tokenizer
        self.prompt = prompt

    def run(self, text):
        full_prompt = f"""
<s>[INST]
{self.prompt}

CONTRACT TEXT:
{text}
[/INST]
"""
        inputs = self.tokenizer(
            full_prompt,
            return_tensors="pt",
            truncation=True,
            max_length=4096
        ).to(self.model.device)

        with torch.no_grad():
            output = self.model.generate(
                **inputs,
                max_new_tokens=600,
                do_sample=False,
                pad_token_id=self.tokenizer.eos_token_id
            )

        # Remove prompt tokens
        generated = output[0][inputs["input_ids"].shape[-1]:]
        return self.tokenizer.decode(
            generated, skip_special_tokens=True
        ).strip()

# -----------------------------
# 7️⃣ RUN OPERATIONS AGENT
# -----------------------------
operations_agent = OperationsAgent(
    model=model,
    tokenizer=tokenizer,
    prompt=OPERATIONS_AGENT_PROMPT
)

raw_output = operations_agent.run(combined_text)
print("\n🔹 RAW OUTPUT (TRUNCATED):\n", raw_output[:800])

# -----------------------------
# 8️⃣ VALIDATE OPERATIONS OUTPUT
# -----------------------------
def extract_json(text):
    text = re.sub(r"```json|```", "", text)
    start = text.find("{")
    end = text.rfind("}") + 1
    return json.loads(text[start:end])

validated_output = extract_json(raw_output)

validated_output.setdefault("extracted_clauses", [])
validated_output.setdefault("risk_level", "medium")
validated_output.setdefault("confidence", 0.8)
validated_output.setdefault("evidence", [])

print("✅ Operations JSON validated")

# -----------------------------
# 9️⃣ SAVE OPERATIONS OUTPUT
# -----------------------------
OUTPUT_FILE = "/content/drive/MyDrive/ClauseAI/data/Agent/operations_agent_output.json"
os.makedirs(os.path.dirname(OUTPUT_FILE), exist_ok=True)

with open(OUTPUT_FILE, "w", encoding="utf-8") as f:
    json.dump(validated_output, f, indent=2, ensure_ascii=False)

print(f"\n✅ Operations Agent output saved at:\n{OUTPUT_FILE}")